# Convolutional Neural Networks with Keras
***

![keras](../keras_logo.png)

***
__Keras is a high level machine learning API built using Python. The library is is running on top of Tensorflow, Theano or CNTK which allows you the choice of what framework backend you want to run your model with.__ 

Throughout each of the tutorials on convolutional neural networks I will be referencing the associated theory document so that you can get a good understanding of what is going on in the background. So, please make sure to read the theory that I reference if you are struggling to understand what is going on. 
***

In [1]:
# Imports
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Activation, Convolution2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.optimizers import Adam, SGD, rmsprop
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


***
## Load In The Data
You will need to download the data from [Kaggle](https://www.kaggle.com/c/dogs-vs-cats/data). The data has been organised into a training and testing folder. Before we can get to work with the data it needs to be organised into folders representing all of the cats and all of the dogs. It's very important that you are able to understand the data that you are working with before building and training the model.
***
In Jupyter Notebooks you can interact directly with the console by using the "!" syntax before your command. It is useful to make use of this to interact with your data directly and find the location of specific files without having to leave the environment. 

In [ ]:
!ls ../../../data/all

When working with file paths a lot, it is good to define them beforehand. We have done that here; a general path, which directs you to all of you downloaded data and a train/test path that points directly at our image data that we want to use. Now we just have to call their set variable names. 

In [ ]:
PATH = "../../../all/"
TRAIN_PATH = "../../../data/all/train/"
TEST_PATH = "../../../data/all/test1/"

Here we set the image height and width that we want all of our images to be. Decreasing the size of our images before we feed them through the model decreases the time it takes to train as there is less data to process. 

```python
IMG_HEIGHT = 256
IMG_WIDTH = 256
```
We then create two arrays which will store our images and their associated class or label, which is found by looking at the file name. 

```python
images = []
classes = []
```
The for loop goes through every image in the training folder and loads each image in individually, appending each to the "images" array. Here we can use the height and width that we set. We then check the filename of the image file that we just loaded in for either cat or dog to set the according number. 

```python
for filename in tqdm(os.listdir(TRAIN_2_PATH)):
    img = mpimg.imread(TRAIN_2_PATH+filename)
    images.append(cv.resize(img, (IMG_HEIGHT, IMG_WIDTH)))
    if "cat" in filename:
        classes.append(0)
    elif "dog" in filename:
        classes.append(1)
```

In [ ]:
IMG_HEIGHT = 256
IMG_WIDTH = 256

images = []
classes = []

for filename in tqdm(os.listdir(TRAIN_PATH)):
    img = mpimg.imread(TRAIN_PATH+filename)
    images.append(cv.resize(img, (IMG_HEIGHT, IMG_WIDTH)))
    if "cat" in filename:
        classes.append(0)
    elif "dog" in filename:
        classes.append(1)

To make sure that we have loaded our images in correctly and our labels for the associated image are correct we can plot the images with their label. We will do this with matplotlib and plot the first six images with their labels as the title. Make sure that the labels are correctly matching the image, 1 for dogs and 0 for cats. 

In [ ]:
COL = 3
ROW = 2
fig = plt.figure(figsize=(8, 8))

for i in range(0, COL*ROW):
    fig.add_subplot(ROW, COL, i+1)          #IF YOU DON'T HAVE +1, IT FALLS OUT OF RANGE
    plt.title(str(classes[i]))
    plt.imshow(images[i])
    
plt.show()

## Augment The Data
It is important that we augment (manipulate) our data to generalise and avoid overfitting. Overfitting happens when our model becomes to acustomed to the features of our training data and so performs worse when validating our performance. By augmenting our data we are generating a set amount of variations for each image that we pass into the `ImageDataGenerator()`. 
***
Set X equal to the "images" array wrapped in another numpy array; we need to do this because the model is expecting a 4 dimensional array. The array it is expecting should look like, `(Total Images, Img Height, Img Width, Img Depth)` when using the `.shape()` function on the array. Y is set equal to the "classes" array as the classes array needs to be in an array of the same dimensions as X.

In [ ]:
X = np.array(images, dtype=np.float32)
Y = np.array(classes, dtype=np.uint8)

Using the scikit-learn library which has a lot of useful functions that we can make use of, we are going to use the `train_test_split` function in particular. The train_test_split function allows us to take our total data to create a training set and a validation set. The training and validation data allows us to see how the model is performing while it is training. 

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.2)

The `ImageDataGenerator()` object gives us the ability to generate variations of our images so that we can generalise our model by introducing more variations on the images that it is looking at and training on. When setting the parameters of the `ImageDataGenerator`, we are selecting the range of how much we want to affect the generated variations of the selected image. 

In [ ]:
train_datagen = ImageDataGenerator(rescale=1. / 255, 
                                   rotation_range=40, 
                                   #width_shift_range=60, 
                                   #height_shift_range=30, 
                                   #brightness_range=10, 
                                   shear_range=10.0, 
                                   #zoom_range=5.0, 
                                   horizontal_flip=True)

valid_datagen = ImageDataGenerator(rescale=1. / 255)

The `.flow()` function is used to generate batches of data using the images and labels. Here we will be generating batch sizes of 125 images and labels. `shuffle` when equal to `True` shuffles the data so that it is not in the set order that it has been downloaded in. 

In [ ]:
train_generator = train_datagen.flow(x_train, y_train, batch_size=125, shuffle=True)
valid_generator = valid_datagen.flow(x_val, y_val, batch_size=125)

## Build The Model

***

![image_processing](../image_input.png)
***

Now that we have collected our data, modified the images and augmented them to increase the reliability of our model. We need to build the model that we will be training on the data that we have prepared. We will build a `Sequential()` model as it is capable for most problems, the alternative is a functional model which allows for a lot more flexibility as you can connect to more than just the previous and next layers. 

In [ ]:
model = Sequential()

The `Convolution2D` layer extracts the features from the data passed in. Imagine we have the 5x5 image (green) and that it is a special case where the the pixels are only 1 or 0. We will take the 3x3 matrix and slide it over the original image by 1 pixel at a time (this is our stride) and multiply the elements and add them to get a final value for the feature map.

![image_processing](../ConvolvedFeat.gif)

ReLU (Rectified Linear Unit) steps are normally introduced after convolution operations in CNN’s. ReLU is applied per pixel and replaces any negative values with a 0 value.

![image_processing](../ReluActivation.png)

The `MaxPooling2D` layer reduces the dimensionality of the feature maps while retaining the important information. We perform pooling to redact the spatial size of the input, this means pooling makes the input representations smaller, reduces network computations which reduces overfitting and makes the network invariant to small transformations or distortions. 

![image_processing](../Pooling.jpeg)

In [ ]:
model.add(Convolution2D(128, (5, 5), input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))

In [ ]:
model.add(Convolution2D(128, (5, 5)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))

model.add(Convolution2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))

The cell below is what is known as a fully connected layer. The fully connected layer implies that every neutron in the previous layer is connected to every neutron in the next layer. The most popular activation function in the output layer is Softmax however other classifiers such as SVM can be used. 

![image_processing](../FullyConnected.png)

The concolution layers above represent the high level features of the data, while adding a fully connected layer is a cheap method of learning a non-linear combination of these features. 

In [ ]:
model.add(Flatten())
model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dense(256))
model.add(Dense(1))
model.add(Activation("sigmoid"))

In [ ]:
model.compile(optimizer="adam", metrics=["accuracy"], loss="binary_crossentropy")

In [ ]:
model.fit_generator(train_generator, 
                    steps_per_epoch=20000 // 125, 
                    epochs=1, 
                    validation_data=valid_generator)